# Snowtornado

In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
products = pd.read_csv("data/cellphone.csv", dtype=str, index_col="id")
matches = pd.read_csv("data/matches_cellphone.csv", index_col=False, usecols=[0, 1], names=['Original', 'Replacement'])

products

,name,E-GSM,Bandtyp,WAP,Antenn,Användarprofiler,Intelligent ordbok,Nätverksanslutning,"Taltid, max",Ir/IrDa,...,Hastighet,Qwerty-tangentbord,4G,Modell,Internminne,Fingeravtrycksläsare,NFC,Vattentät,Upplösningsformat,Välvd
id,,,,,,,,,,,,,,,,,,,,,
618,C25,Ja,Dualband,Nej,Fast,Nej,Nej,Nej,5,Nej,...,NaN,Nej,Nej,NaN,NaN,Nej,Nej,Nej,< HD,Nej
619,Savvy DB,Nej,Dualband,Nej,Fast,Nej,Nej,Nej,4,Nej,...,NaN,Nej,Nej,NaN,NaN,Nej,Nej,Nej,< HD,Nej
620,SL10,Nej,Singleband,Nej,Fast,Nej,Nej,Ja,3,Ja,...,NaN,Nej,Nej,NaN,NaN,Nej,Nej,Nej,< HD,Nej
621,C28,Ja,Dualband,Nej,Fast,Ja,Nej,Nej,5,Nej,...,NaN,Nej,Nej,NaN,NaN,Nej,Nej,Nej,< HD,Nej
622,S10 Active,Nej,Singleband,Nej,Utdragbar,Nej,Nej,Nej,10,Nej,...,NaN,Nej,Nej,NaN,NaN,Nej,Nej,Nej,< HD,Nej
623,E10,Nej,Singleband,Nej,Fast,Nej,Nej,Ja,6,Nej,...,NaN,Nej,Nej,NaN,NaN,Nej,Nej,Nej,< HD,Nej
624,S25,Ja,Dualband,Nej,Fast,Ja,Nej,Ja,5,Ja,...,NaN,Nej,Nej,NaN,NaN,Nej,Nej,Nej,< HD,Nej
625,S35i,Ja,Dualband,Ja,Integrerad,Ja,Ja,Ja,6,Ja,...,NaN,Nej,Nej,NaN,NaN,Nej,Nej,Nej,< HD,Nej
626,M35i,Ja,Dualband,Ja,Integrerad,Ja,Ja,Ja,5,Nej,...,NaN,Nej,Nej,NaN,NaN,Nej,Nej,Nej,< HD,Nej


Clean data by converting every field to lowercase, stripping, and finally converting it to a category type

In [3]:
binary = pd.DataFrame()

for column in products.columns:
    if not column in ["name"]:
        binary[column] = products[column].str.lower().str.strip().astype('category')
    
binary = pd.get_dummies(binary, dummy_na=True)
binary = binary.loc[:, (binary != 0).any(axis=0)] # Remove superfluous NaN columns
binary.shape

(1876, 679)

In [4]:
matches_train, matches_test = train_test_split(matches, test_size=0.3, random_state=42)

Fill training data with the binary data

In [5]:
X_train = np.zeros((matches_train.shape[0], binary.shape[1]), dtype = 'bool')
X_test = np.zeros((matches_test.shape[0], binary.shape[1]), dtype = 'bool')
Y_train = np.zeros((matches_train.shape[0], binary.shape[1]), dtype = 'bool')
Y_test = np.zeros((matches_test.shape[0], binary.shape[1]), dtype = 'bool')

In [6]:
i = 0
for _, row in matches_train.iterrows():
    original_id = row['Original']
    replacement_id = row['Replacement']
    X_train[i] = binary.loc[[original_id]].values[0]
    Y_train[i] = binary.loc[[replacement_id]].values[0]
    i += 1

i = 0
for _, row in matches_test.iterrows():
    original_id = row['Original']
    replacement_id = row['Replacement']
    X_test[i] = binary.loc[[original_id]].values[0]
    Y_test[i] = binary.loc[[replacement_id]].values[0]
    i += 1

In [7]:
# The following model could still be supplemanted by something like a tf-idf preprocess. 
model = DecisionTreeRegressor(max_depth=4).fit(X_train, Y_train)
model.score(X_test, Y_test)

0.24850450153615414

In [8]:
# Maybe the scoring can be improved upon? What we want is not R^2 but the correct 'directions'
# If cosine similarity is what is going to be used then that could be used to determine the 
# performance of the algorithm?
#np.square(cosine_similarity(model.predict(X_test), Y_test) - 1).sum() / X_test.shape[0]
cosine_similarity(model.predict(X_test), Y_test).mean()

0.8064906964780395

In [9]:
def find_matches(original_id, limit = 3):
    prediction = model.predict(binary.loc[[original_id]].values)
    similar = cosine_similarity(prediction, binary)[0]
    indices = np.argpartition(similar, -limit)[-limit:]
    indices = indices[np.argsort(similar[indices])][::-1]
    return binary.index.values[indices]

products.loc[find_matches(38096, 10)]

,name,E-GSM,Bandtyp,WAP,Antenn,Användarprofiler,Intelligent ordbok,Nätverksanslutning,"Taltid, max",Ir/IrDa,...,Hastighet,Qwerty-tangentbord,4G,Modell,Internminne,Fingeravtrycksläsare,NFC,Vattentät,Upplösningsformat,Välvd
id,,,,,,,,,,,,,,,,,,,,,
66270,iPhone 5c 16GB,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1300,Nej,Ja,A6,1024,Nej,Nej,Nej,< HD,Nej
66272,iPhone 5c 32GB,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1300,Nej,Ja,A6,1024,Nej,Nej,Nej,< HD,Nej
66273,iPhone 5s 16GB,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1300,Nej,Ja,A7,1024,Ja,Nej,Nej,< HD,Nej
66267,iPhone 5s 64GB,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1300,Nej,Ja,A7,1024,Ja,Nej,Nej,< HD,Nej
66274,iPhone 5s 32GB,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1300,Nej,Ja,A7,1024,Ja,Nej,Nej,< HD,Nej
71988,iPhone 5c 8GB,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1300,Nej,Ja,A6,1024,Nej,Nej,Nej,< HD,Nej
52738,iPhone 5 16GB,NaN,Quad-Band,Nej,NaN,NaN,NaN,NaN,8,NaN,...,1200,Nej,Ja,A6,1024,Nej,Nej,Nej,< HD,Nej
65952,iPhone 4s 8GB,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,800,Nej,Nej,A5,512,Nej,Nej,Nej,< HD,Nej
52742,iPhone 5 64GB,NaN,Quad-Band,Nej,NaN,NaN,NaN,NaN,8,NaN,...,1200,Nej,Ja,A6,1024,Nej,Nej,Nej,< HD,Nej


In [10]:
from IPython.display import display, HTML
pd.options.display.max_columns = 100
from itertools import islice

for _, row in islice(matches_test.iterrows(), 10):
    original_id = row['Original']
    replacement_id = row['Replacement']
    
    display(products.loc[[original_id, replacement_id, find_matches(original_id, 1)[0]]])


,name,E-GSM,Bandtyp,WAP,Antenn,Användarprofiler,Intelligent ordbok,Nätverksanslutning,"Taltid, max",Ir/IrDa,SMS,Vibrator,Valutaomvandlare,Alarm,Tid/Datum,FM-Radio,Röststyrning,Mp3/WMA,Kalkylator,Kalender,Spel,Röstmemo,Bluetooth,Telefonbok,Högtalarfunktion,Kamera,"Standbytid, max",GPRS,HSCSD,Mp3,MMS,EMS,E-mail funktion,Java,Ringsignaler,3G,"Video,uppspelning",USB,Pictbridge,Max upplösning (stillbild),Max upplösning (video),Storlek,Färgsupport,Bildskärmsteknik,Pixlar,Telefondesign,Bildskärmsteknik.1,Pixlar.1,Lagringskapacitet,Kortminne,EDGE,Trådlöst nätverk,Pekskärm,Operativsystem,GPS,HSDPA,Kortplats,Megapixlar,3D fotografering,Antal kärnor,Processortyp,HSPA+,Hastighet,Qwerty-tangentbord,4G,Modell,Internminne,Fingeravtrycksläsare,NFC,Vattentät,Upplösningsformat,Välvd
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
52556,Xperia Go,NaN,Quad-Band,Nej,NaN,NaN,NaN,NaN,6.5,NaN,NaN,Nej,NaN,NaN,NaN,Ja,Ja,NaN,NaN,NaN,NaN,Ja,Ja,NaN,Ja,Ja,460,Ja,NaN,Ja,Ja,NaN,Ja,Ja,NaN,Ja,NaN,Ja,Nej,2592x1944,1280x720,3.5,16000000,NaN,NaN,Standard,LCD,320x480,4,NaN,Ja,Ja,Ja,Android,Ja,Ja,Ja,5,Nej,2,ARM Cortex-A9,Nej,1000,Nej,Nej,NovaThor,512,Nej,Nej,Ja,< HD,Nej
69445,Trooper X3.5,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,NaN,Ja,NaN,NaN,NaN,NaN,NaN,Ja,Ja,NaN,NaN,Ja,NaN,NaN,NaN,Ja,NaN,NaN,NaN,Ja,NaN,Ja,Ja,Ja,NaN,640x480,1280x720,3.5,16000000,NaN,NaN,Standard,LCD,320x480,4,NaN,NaN,Ja,Ja,Android,Ja,NaN,Ja,3.2,NaN,2,ARM Cortex-A7,NaN,1000,Nej,Nej,NaN,512,Nej,Nej,Nej,< HD,Nej
74026,Galaxy Ace Style G310,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,Ja,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,Ja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,NaN,Ja,NaN,Ja,NaN,2592x1944,1280x720,4,16000000,NaN,NaN,Standard,LCD,800x480,4,NaN,NaN,Ja,Ja,Android,Ja,NaN,Ja,5,NaN,2,NaN,NaN,1200,Nej,Nej,NaN,512,Nej,Ja,Nej,< HD,Nej


,name,E-GSM,Bandtyp,WAP,Antenn,Användarprofiler,Intelligent ordbok,Nätverksanslutning,"Taltid, max",Ir/IrDa,SMS,Vibrator,Valutaomvandlare,Alarm,Tid/Datum,FM-Radio,Röststyrning,Mp3/WMA,Kalkylator,Kalender,Spel,Röstmemo,Bluetooth,Telefonbok,Högtalarfunktion,Kamera,"Standbytid, max",GPRS,HSCSD,Mp3,MMS,EMS,E-mail funktion,Java,Ringsignaler,3G,"Video,uppspelning",USB,Pictbridge,Max upplösning (stillbild),Max upplösning (video),Storlek,Färgsupport,Bildskärmsteknik,Pixlar,Telefondesign,Bildskärmsteknik.1,Pixlar.1,Lagringskapacitet,Kortminne,EDGE,Trådlöst nätverk,Pekskärm,Operativsystem,GPS,HSDPA,Kortplats,Megapixlar,3D fotografering,Antal kärnor,Processortyp,HSPA+,Hastighet,Qwerty-tangentbord,4G,Modell,Internminne,Fingeravtrycksläsare,NFC,Vattentät,Upplösningsformat,Välvd
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
84280,Galaxy S6 Edge+ G928F 32GB,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nej,Ja,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,Ja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,NaN,Ja,NaN,Ja,NaN,NaN,4096x2160,5.7,16000000,NaN,NaN,Standard,OLED,2560x1440,32,NaN,NaN,Ja,Ja,Android,Ja,NaN,Nej,16,NaN,4,NaN,NaN,2100,Nej,Ja,NaN,4096,Ja,Ja,Nej,QUAD HD,Nej
84281,Galaxy S6 Edge+ G928F 64GB,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nej,Ja,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,Ja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,NaN,Ja,NaN,Ja,NaN,NaN,4096x2160,5.7,16000000,NaN,NaN,Standard,OLED,2560x1440,64,NaN,NaN,Ja,Ja,Android,Ja,NaN,Nej,16,NaN,4,NaN,NaN,2100,Nej,Ja,NaN,4096,Ja,Ja,Nej,QUAD HD,Nej
81174,Galaxy S6 G920F 32GB,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nej,Ja,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,Ja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,NaN,Ja,NaN,Ja,NaN,NaN,3840x2160,5.1,16000000,NaN,NaN,Standard,OLED,2560x1440,32,NaN,NaN,Ja,Ja,Android,Ja,NaN,Nej,16,NaN,4,NaN,NaN,2100,Nej,Ja,NaN,3072,Ja,Ja,Nej,QUAD HD,Nej


,name,E-GSM,Bandtyp,WAP,Antenn,Användarprofiler,Intelligent ordbok,Nätverksanslutning,"Taltid, max",Ir/IrDa,SMS,Vibrator,Valutaomvandlare,Alarm,Tid/Datum,FM-Radio,Röststyrning,Mp3/WMA,Kalkylator,Kalender,Spel,Röstmemo,Bluetooth,Telefonbok,Högtalarfunktion,Kamera,"Standbytid, max",GPRS,HSCSD,Mp3,MMS,EMS,E-mail funktion,Java,Ringsignaler,3G,"Video,uppspelning",USB,Pictbridge,Max upplösning (stillbild),Max upplösning (video),Storlek,Färgsupport,Bildskärmsteknik,Pixlar,Telefondesign,Bildskärmsteknik.1,Pixlar.1,Lagringskapacitet,Kortminne,EDGE,Trådlöst nätverk,Pekskärm,Operativsystem,GPS,HSDPA,Kortplats,Megapixlar,3D fotografering,Antal kärnor,Processortyp,HSPA+,Hastighet,Qwerty-tangentbord,4G,Modell,Internminne,Fingeravtrycksläsare,NFC,Vattentät,Upplösningsformat,Välvd
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
81829,Xperia T3,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,Nej,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,Ja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,NaN,Ja,NaN,Ja,NaN,3264x2448,1920x1080,5.3,16000000,NaN,NaN,Standard,LCD,1280x720,8,NaN,NaN,Ja,Ja,Android,Ja,NaN,Ja,8,NaN,4,ARM Cortex-A7,NaN,1400,Nej,Ja,Snapdragon S4,1024,Nej,Ja,Nej,HD,Nej
88667,Xperia XA,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,Ja,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,Ja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,NaN,Ja,NaN,Ja,NaN,NaN,1920x1080,5,16777216,NaN,NaN,Standard,LCD,1280x720,16,NaN,NaN,Ja,Ja,Android,Ja,NaN,Ja,13,NaN,8,NaN,NaN,2000,Nej,Ja,NaN,2048,Ja,Ja,Nej,FULL HD,Nej
72048,Galaxy S4 Black Edition i9506 16GB,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,Ja,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,Ja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,NaN,Ja,NaN,Ja,NaN,4128x3096,1920x1080,5,16000000,NaN,NaN,Standard,OLED,1920x1080,16,NaN,NaN,Ja,Ja,Android,Ja,NaN,Ja,13,NaN,4,Krait,NaN,2300,Nej,Ja,Snapdragon 800,2048,Nej,Ja,Nej,FULL HD,Nej


,name,E-GSM,Bandtyp,WAP,Antenn,Användarprofiler,Intelligent ordbok,Nätverksanslutning,"Taltid, max",Ir/IrDa,SMS,Vibrator,Valutaomvandlare,Alarm,Tid/Datum,FM-Radio,Röststyrning,Mp3/WMA,Kalkylator,Kalender,Spel,Röstmemo,Bluetooth,Telefonbok,Högtalarfunktion,Kamera,"Standbytid, max",GPRS,HSCSD,Mp3,MMS,EMS,E-mail funktion,Java,Ringsignaler,3G,"Video,uppspelning",USB,Pictbridge,Max upplösning (stillbild),Max upplösning (video),Storlek,Färgsupport,Bildskärmsteknik,Pixlar,Telefondesign,Bildskärmsteknik.1,Pixlar.1,Lagringskapacitet,Kortminne,EDGE,Trådlöst nätverk,Pekskärm,Operativsystem,GPS,HSDPA,Kortplats,Megapixlar,3D fotografering,Antal kärnor,Processortyp,HSPA+,Hastighet,Qwerty-tangentbord,4G,Modell,Internminne,Fingeravtrycksläsare,NFC,Vattentät,Upplösningsformat,Välvd
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
44287,iPhone 4s 16GB,NaN,Quad-Band,Ja,Integrerad,Ja,NaN,NaN,14,NaN,NaN,Ja,NaN,NaN,NaN,Nej,Ja,NaN,NaN,NaN,NaN,Ja,Ja,NaN,Ja,Ja,200,Ja,NaN,Ja,Ja,NaN,Ja,Nej,Polyfoniska. MP3,Ja,Ja,Ja,Nej,3264x2448,1920x1080,3.5,16000000,NaN,NaN,Standard,LCD,960x640,16,NaN,Ja,Ja,Ja,iOS,Ja,Ja,Nej,8,NaN,2,ARM Cortex-A9,NaN,800,Nej,Nej,A5,512,Nej,Nej,Nej,< HD,Nej
66273,iPhone 5s 16GB,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,NaN,Nej,Ja,NaN,NaN,NaN,NaN,Ja,Ja,NaN,NaN,Ja,NaN,NaN,NaN,Ja,NaN,NaN,NaN,Nej,NaN,Ja,Ja,Nej,NaN,3264x2448,1920x1080,4,16000000,NaN,NaN,Standard,LCD,1136x640,16,NaN,NaN,Ja,Ja,iOS,Ja,NaN,Nej,8,Nej,2,NaN,NaN,1300,Nej,Ja,A7,1024,Ja,Nej,Nej,< HD,Nej
66273,iPhone 5s 16GB,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,NaN,Nej,Ja,NaN,NaN,NaN,NaN,Ja,Ja,NaN,NaN,Ja,NaN,NaN,NaN,Ja,NaN,NaN,NaN,Nej,NaN,Ja,Ja,Nej,NaN,3264x2448,1920x1080,4,16000000,NaN,NaN,Standard,LCD,1136x640,16,NaN,NaN,Ja,Ja,iOS,Ja,NaN,Nej,8,Nej,2,NaN,NaN,1300,Nej,Ja,A7,1024,Ja,Nej,Nej,< HD,Nej


,name,E-GSM,Bandtyp,WAP,Antenn,Användarprofiler,Intelligent ordbok,Nätverksanslutning,"Taltid, max",Ir/IrDa,SMS,Vibrator,Valutaomvandlare,Alarm,Tid/Datum,FM-Radio,Röststyrning,Mp3/WMA,Kalkylator,Kalender,Spel,Röstmemo,Bluetooth,Telefonbok,Högtalarfunktion,Kamera,"Standbytid, max",GPRS,HSCSD,Mp3,MMS,EMS,E-mail funktion,Java,Ringsignaler,3G,"Video,uppspelning",USB,Pictbridge,Max upplösning (stillbild),Max upplösning (video),Storlek,Färgsupport,Bildskärmsteknik,Pixlar,Telefondesign,Bildskärmsteknik.1,Pixlar.1,Lagringskapacitet,Kortminne,EDGE,Trådlöst nätverk,Pekskärm,Operativsystem,GPS,HSDPA,Kortplats,Megapixlar,3D fotografering,Antal kärnor,Processortyp,HSPA+,Hastighet,Qwerty-tangentbord,4G,Modell,Internminne,Fingeravtrycksläsare,NFC,Vattentät,Upplösningsformat,Välvd
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
46981,Xperia Arc S,NaN,Quad-Band,Ja,Integrerad,Ja,NaN,NaN,7.5,NaN,NaN,Ja,NaN,NaN,NaN,Ja,Ja,NaN,NaN,NaN,NaN,Ja,Ja,NaN,Ja,Ja,450,Ja,NaN,Ja,Ja,NaN,Ja,Ja,Polyfoniska. MP3,Ja,Ja,Ja,Nej,3264x2448,1280x720,4.2,16000000,NaN,NaN,Standard,LCD,854x480,1,8192,Ja,Ja,Ja,Android,Ja,Ja,Ja,8,Ja,1,Scorpion,NaN,1400,Nej,Nej,Snapdragon S2,512,Nej,Nej,Nej,< HD,Nej
58674,Xperia L,NaN,Quad-Band,Nej,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,NaN,Ja,Nej,NaN,NaN,NaN,NaN,Ja,Ja,NaN,NaN,Ja,NaN,Ja,NaN,Ja,NaN,NaN,NaN,Ja,NaN,Ja,Nej,Ja,Nej,3264x2448,1280x720,4.3,16000000,NaN,NaN,Standard,LCD,854x480,8,NaN,Ja,Ja,Ja,Android,Ja,Ja,Ja,8,Nej,2,Krait,Nej,1000,Nej,Nej,Snapdragon S4,1024,Nej,Ja,Nej,< HD,Nej
74026,Galaxy Ace Style G310,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,Ja,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,Ja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,NaN,Ja,NaN,Ja,NaN,2592x1944,1280x720,4,16000000,NaN,NaN,Standard,LCD,800x480,4,NaN,NaN,Ja,Ja,Android,Ja,NaN,Ja,5,NaN,2,NaN,NaN,1200,Nej,Nej,NaN,512,Nej,Ja,Nej,< HD,Nej


,name,E-GSM,Bandtyp,WAP,Antenn,Användarprofiler,Intelligent ordbok,Nätverksanslutning,"Taltid, max",Ir/IrDa,SMS,Vibrator,Valutaomvandlare,Alarm,Tid/Datum,FM-Radio,Röststyrning,Mp3/WMA,Kalkylator,Kalender,Spel,Röstmemo,Bluetooth,Telefonbok,Högtalarfunktion,Kamera,"Standbytid, max",GPRS,HSCSD,Mp3,MMS,EMS,E-mail funktion,Java,Ringsignaler,3G,"Video,uppspelning",USB,Pictbridge,Max upplösning (stillbild),Max upplösning (video),Storlek,Färgsupport,Bildskärmsteknik,Pixlar,Telefondesign,Bildskärmsteknik.1,Pixlar.1,Lagringskapacitet,Kortminne,EDGE,Trådlöst nätverk,Pekskärm,Operativsystem,GPS,HSDPA,Kortplats,Megapixlar,3D fotografering,Antal kärnor,Processortyp,HSPA+,Hastighet,Qwerty-tangentbord,4G,Modell,Internminne,Fingeravtrycksläsare,NFC,Vattentät,Upplösningsformat,Välvd
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
82367,Honor 4X,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,Ja,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,Ja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,NaN,Ja,NaN,Ja,NaN,4208x3120,1920x1080,5.5,16000000,NaN,NaN,Standard,LCD,1280x800,8,NaN,NaN,Ja,Ja,Android,Ja,NaN,Ja,13,NaN,4,NaN,NaN,1200,Nej,Ja,NaN,2048,Nej,Nej,Nej,HD,Nej
88253,Honor 5X,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,Ja,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,Ja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,NaN,Ja,NaN,Ja,NaN,4208x3120,1920x1080,5.5,16777216,NaN,NaN,Standard,LCD,1920x1080,16,NaN,NaN,Ja,Ja,Android,Ja,NaN,Ja,13,NaN,8,NaN,NaN,1500,Nej,Ja,NaN,2048,Nej,Nej,Nej,FULL HD,Nej
72048,Galaxy S4 Black Edition i9506 16GB,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,Ja,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,Ja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,NaN,Ja,NaN,Ja,NaN,4128x3096,1920x1080,5,16000000,NaN,NaN,Standard,OLED,1920x1080,16,NaN,NaN,Ja,Ja,Android,Ja,NaN,Ja,13,NaN,4,Krait,NaN,2300,Nej,Ja,Snapdragon 800,2048,Nej,Ja,Nej,FULL HD,Nej


,name,E-GSM,Bandtyp,WAP,Antenn,Användarprofiler,Intelligent ordbok,Nätverksanslutning,"Taltid, max",Ir/IrDa,SMS,Vibrator,Valutaomvandlare,Alarm,Tid/Datum,FM-Radio,Röststyrning,Mp3/WMA,Kalkylator,Kalender,Spel,Röstmemo,Bluetooth,Telefonbok,Högtalarfunktion,Kamera,"Standbytid, max",GPRS,HSCSD,Mp3,MMS,EMS,E-mail funktion,Java,Ringsignaler,3G,"Video,uppspelning",USB,Pictbridge,Max upplösning (stillbild),Max upplösning (video),Storlek,Färgsupport,Bildskärmsteknik,Pixlar,Telefondesign,Bildskärmsteknik.1,Pixlar.1,Lagringskapacitet,Kortminne,EDGE,Trådlöst nätverk,Pekskärm,Operativsystem,GPS,HSDPA,Kortplats,Megapixlar,3D fotografering,Antal kärnor,Processortyp,HSPA+,Hastighet,Qwerty-tangentbord,4G,Modell,Internminne,Fingeravtrycksläsare,NFC,Vattentät,Upplösningsformat,Välvd
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
71382,Nexus 4 E960 8GB,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,NaN,Ja,Ja,NaN,NaN,NaN,NaN,Ja,Ja,NaN,NaN,Ja,NaN,NaN,NaN,Ja,NaN,NaN,NaN,Ja,NaN,Ja,Ja,Ja,NaN,3264x2448,1920x1080,4.7,16000000,NaN,NaN,Standard,LCD,1280x768,8,NaN,NaN,Ja,Ja,Android,Ja,NaN,Nej,8,NaN,4,Krait,NaN,1500,Nej,Nej,Snapdragon S4,2048,Nej,Ja,Nej,< HD,Nej
62211,Ascend P6,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,NaN,Ja,Ja,NaN,NaN,NaN,NaN,Ja,Ja,NaN,NaN,Ja,NaN,NaN,NaN,Ja,NaN,NaN,NaN,Ja,NaN,Ja,Ja,Ja,NaN,3264x2448,1280x720,4.7,16000000,NaN,NaN,Standard,LCD,1280x720,8,NaN,NaN,Ja,Ja,Android,Ja,NaN,Ja,8,Nej,4,NaN,NaN,1500,Nej,Nej,NaN,2048,Nej,Ja,Nej,HD,Nej
74026,Galaxy Ace Style G310,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,Ja,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,Ja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,NaN,Ja,NaN,Ja,NaN,2592x1944,1280x720,4,16000000,NaN,NaN,Standard,LCD,800x480,4,NaN,NaN,Ja,Ja,Android,Ja,NaN,Ja,5,NaN,2,NaN,NaN,1200,Nej,Nej,NaN,512,Nej,Ja,Nej,< HD,Nej


,name,E-GSM,Bandtyp,WAP,Antenn,Användarprofiler,Intelligent ordbok,Nätverksanslutning,"Taltid, max",Ir/IrDa,SMS,Vibrator,Valutaomvandlare,Alarm,Tid/Datum,FM-Radio,Röststyrning,Mp3/WMA,Kalkylator,Kalender,Spel,Röstmemo,Bluetooth,Telefonbok,Högtalarfunktion,Kamera,"Standbytid, max",GPRS,HSCSD,Mp3,MMS,EMS,E-mail funktion,Java,Ringsignaler,3G,"Video,uppspelning",USB,Pictbridge,Max upplösning (stillbild),Max upplösning (video),Storlek,Färgsupport,Bildskärmsteknik,Pixlar,Telefondesign,Bildskärmsteknik.1,Pixlar.1,Lagringskapacitet,Kortminne,EDGE,Trådlöst nätverk,Pekskärm,Operativsystem,GPS,HSDPA,Kortplats,Megapixlar,3D fotografering,Antal kärnor,Processortyp,HSPA+,Hastighet,Qwerty-tangentbord,4G,Modell,Internminne,Fingeravtrycksläsare,NFC,Vattentät,Upplösningsformat,Välvd
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
38096,iPhone 4 32GB,NaN,Quad-Band,Ja,Integrerad,Ja,NaN,Nej,14,Nej,NaN,Ja,NaN,NaN,NaN,Nej,Ja,NaN,NaN,NaN,NaN,Ja,Ja,NaN,Ja,Ja,300,Ja,NaN,Ja,Ja,Nej,Ja,Nej,Polyfoniska. MP3,Ja,Ja,Ja,Nej,2592x1944,1280x720,3.5,16000000,NaN,NaN,Standard,LCD,960x640,32,NaN,Ja,Ja,Ja,iOS,Ja,Ja,Nej,5,NaN,1,ARM Cortex-A8,NaN,800,Nej,Nej,A4,512,Nej,Nej,Nej,< HD,Nej
71988,iPhone 5c 8GB,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nej,Ja,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,Ja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nej,NaN,Ja,NaN,Nej,NaN,3264x2448,1920x1080,4,16000000,NaN,NaN,Standard,LCD,1136x640,8,NaN,NaN,Ja,Ja,iOS,Ja,NaN,Nej,8,NaN,2,ARM Cortex-A7,NaN,1300,Nej,Ja,A6,1024,Nej,Nej,Nej,< HD,Nej
66270,iPhone 5c 16GB,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,NaN,Nej,Ja,NaN,NaN,NaN,NaN,Ja,Ja,NaN,NaN,Ja,NaN,NaN,NaN,Ja,NaN,NaN,NaN,Nej,NaN,Ja,Ja,Nej,NaN,3264x2448,1920x1080,4,16000000,NaN,NaN,Standard,LCD,1136x640,16,NaN,NaN,Ja,Ja,iOS,Ja,NaN,Nej,8,Nej,2,ARM Cortex-A7,NaN,1300,Nej,Ja,A6,1024,Nej,Nej,Nej,< HD,Nej


,name,E-GSM,Bandtyp,WAP,Antenn,Användarprofiler,Intelligent ordbok,Nätverksanslutning,"Taltid, max",Ir/IrDa,SMS,Vibrator,Valutaomvandlare,Alarm,Tid/Datum,FM-Radio,Röststyrning,Mp3/WMA,Kalkylator,Kalender,Spel,Röstmemo,Bluetooth,Telefonbok,Högtalarfunktion,Kamera,"Standbytid, max",GPRS,HSCSD,Mp3,MMS,EMS,E-mail funktion,Java,Ringsignaler,3G,"Video,uppspelning",USB,Pictbridge,Max upplösning (stillbild),Max upplösning (video),Storlek,Färgsupport,Bildskärmsteknik,Pixlar,Telefondesign,Bildskärmsteknik.1,Pixlar.1,Lagringskapacitet,Kortminne,EDGE,Trådlöst nätverk,Pekskärm,Operativsystem,GPS,HSDPA,Kortplats,Megapixlar,3D fotografering,Antal kärnor,Processortyp,HSPA+,Hastighet,Qwerty-tangentbord,4G,Modell,Internminne,Fingeravtrycksläsare,NFC,Vattentät,Upplösningsformat,Välvd
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
83050,Galaxy S4 i9505 64GB,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,Ja,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,Ja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,NaN,Ja,NaN,Ja,NaN,4128x3096,1920x1080,5,16000000,NaN,NaN,Standard,OLED,1920x1080,64,NaN,NaN,Ja,Ja,Android,Ja,NaN,Ja,13,NaN,4,Krait,NaN,1900,Nej,Ja,Snapdragon 600,2048,Nej,Ja,Nej,FULL HD,Nej
81175,Galaxy S6 G920F 64GB,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nej,Ja,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,Ja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,NaN,Ja,NaN,Ja,NaN,NaN,3840x2160,5.1,16000000,NaN,NaN,Standard,OLED,2560x1440,64,NaN,NaN,Ja,Ja,Android,Ja,NaN,Nej,16,NaN,4,NaN,NaN,2100,Nej,Ja,NaN,3072,Ja,Ja,Nej,QUAD HD,Nej
72048,Galaxy S4 Black Edition i9506 16GB,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,Ja,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,Ja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,NaN,Ja,NaN,Ja,NaN,4128x3096,1920x1080,5,16000000,NaN,NaN,Standard,OLED,1920x1080,16,NaN,NaN,Ja,Ja,Android,Ja,NaN,Ja,13,NaN,4,Krait,NaN,2300,Nej,Ja,Snapdragon 800,2048,Nej,Ja,Nej,FULL HD,Nej


,name,E-GSM,Bandtyp,WAP,Antenn,Användarprofiler,Intelligent ordbok,Nätverksanslutning,"Taltid, max",Ir/IrDa,SMS,Vibrator,Valutaomvandlare,Alarm,Tid/Datum,FM-Radio,Röststyrning,Mp3/WMA,Kalkylator,Kalender,Spel,Röstmemo,Bluetooth,Telefonbok,Högtalarfunktion,Kamera,"Standbytid, max",GPRS,HSCSD,Mp3,MMS,EMS,E-mail funktion,Java,Ringsignaler,3G,"Video,uppspelning",USB,Pictbridge,Max upplösning (stillbild),Max upplösning (video),Storlek,Färgsupport,Bildskärmsteknik,Pixlar,Telefondesign,Bildskärmsteknik.1,Pixlar.1,Lagringskapacitet,Kortminne,EDGE,Trådlöst nätverk,Pekskärm,Operativsystem,GPS,HSDPA,Kortplats,Megapixlar,3D fotografering,Antal kärnor,Processortyp,HSPA+,Hastighet,Qwerty-tangentbord,4G,Modell,Internminne,Fingeravtrycksläsare,NFC,Vattentät,Upplösningsformat,Välvd
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
52623,Xperia V,NaN,Quad-Band,Nej,NaN,NaN,NaN,NaN,7,NaN,NaN,Ja,NaN,NaN,NaN,Ja,Ja,NaN,NaN,NaN,NaN,Ja,Ja,NaN,Ja,Ja,400,Ja,NaN,Ja,Ja,NaN,Ja,Ja,NaN,Ja,Ja,Ja,Nej,4128x3096,1920x1080,4.3,16000000,NaN,NaN,Standard,LCD,1280x720,8,NaN,Ja,Ja,Ja,Android,Ja,Ja,Ja,13,Nej,2,Krait,Ja,1500,Nej,Ja,Snapdragon S4,1024,Nej,Ja,Ja,HD,Nej
75133,Moto G LTE 8GB,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,Ja,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,Ja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,NaN,Ja,NaN,Ja,NaN,2592x1944,1280x720,4.5,16000000,NaN,NaN,Standard,LCD,1280x720,8,NaN,NaN,Ja,Ja,Android,Ja,NaN,Ja,5,NaN,4,NaN,NaN,1200,Nej,Ja,Snapdragon 400,1024,Nej,Nej,Nej,HD,Nej
74498,Galaxy Note 3 Neo N7505 16GB,NaN,Quad-Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,Ja,NaN,NaN,NaN,NaN,NaN,Ja,NaN,NaN,Ja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,NaN,Ja,NaN,Ja,NaN,3264x2448,1920x1080,5.5,16000000,NaN,NaN,Standard,OLED,1280x720,16,NaN,NaN,Ja,Ja,Android,Ja,NaN,Ja,8,NaN,4,ARM Cortex-A7,NaN,1300,Nej,Ja,Snapdragon 800,2048,Nej,Ja,Nej,HD,Nej
